In [1]:
import os
import numpy as np
import pandas as pd
import hickle as hkl

from collections import defaultdict

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, explained_variance_score, mean_absolute_error, mean_squared_error

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, Dropout, LSTM, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

from IPython.core.display import display, HTML



# Hide GPU from visible devices

#'''
tf.config.set_visible_devices([], 'GPU')
print(f'CUDA GPU AVAILABLE: {tf.test.is_gpu_available(cuda_only=True)}')
'''
THREADS = 8
os.environ['OMP_NUM_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTEROP_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.set_soft_device_placement(True)
'''

#'''
display(HTML("<style>.container { width:100% !important; }</style>"))
#'''

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CUDA GPU AVAILABLE: True


In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true)

In [3]:
N_SPLITS = 5
N_STEPS = 20
N_DAYS = 60
DATA_DIR = f'data/processed_data/cv_data__{N_SPLITS}_class_id_folds__{N_STEPS}_steps__{N_DAYS}_days'

In [4]:
results = []

for i in range(N_SPLITS):

    # Load one partition of folds
    data = hkl.load(f'{DATA_DIR}/fold_{i}.hkl')
    #action_training_input = data['action_training_input']
    recurrent_training_input = data['recurrent_training_input']
    #prior_training_input = data['prior_training_input']
    completion_training_target = data['completion_training_target']
    problems_training_target = data['problems_training_target']
    training_target_sequence = data['training_target_sequence']
    #action_testing_input = data['action_testing_input']
    recurrent_testing_input = data['recurrent_testing_input']
    #prior_testing_input = data['prior_testing_input']
    completion_testing_target = data['completion_testing_target']
    problems_testing_target = data['problems_testing_target']
    testing_target_sequence = data['testing_target_sequence']
    
    # Clear session so models don't pile up
    keras.backend.clear_session()

    # Create model
    
    # Action Model
    #action_input_layer = Input(shape=action_training_input[0].shape, name='action')
    #action_model = Dropout(rate=0.5)(action_input_layer)
    #action_model = LSTM(units=64, return_sequences=False, activation='relu', dropout=0.5, recurrent_dropout=0.5)(action_model)
    
    # Recurrent Input
    recurrent_input_layer = Input(shape=recurrent_training_input[0].shape, name='recurrent')
    recurrent_model = Masking(mask_value=0.0)(recurrent_input_layer)
    recurrent_model = Dropout(rate=0.5)(recurrent_model)
    recurrent_model = LSTM(units=64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)(recurrent_model)
    
    # Prior Input
    #prior_input_layer = Input(shape=prior_training_input[0].shape, name='prior')
    #prior_model = Dropout(rate=0.5)(prior_input_layer)
    #prior_model = Dense(units=64, activation='sigmoid')(prior_model)
    
    # Combined Model
    #model = Concatenate()([action_model, recurrent_model, prior_model])
    model = Dropout(rate=0.5)(recurrent_model)
    
    # Combined Output
    completion_output_layer = Dense(units=1, activation='sigmoid', name='completion')(model)
    problems_output_layer = Dense(units=1, activation='linear', name='problems')(model)
    
    combined_model = Model(recurrent_input_layer, [completion_output_layer, problems_output_layer])
    combined_model.compile(optimizer='adam', loss={'completion': 'binary_crossentropy', 'problems': 'mse'})

    # Train model
    es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]
    weights = {'completion': np.ones_like(completion_training_target) * 16, 'problems': completion_training_target}
    combined_model.fit(x={'recurrent': recurrent_training_input},
                       y={'completion': completion_training_target, 'problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    # Store model predictions
    completion_testing_output, problems_testing_output = combined_model.predict({'recurrent': recurrent_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    

pd.concat(results).to_csv('assignments_cross_validation_results.csv', index=False)

Epoch 1/1000
3949/3949 [==============================] - 192s 48ms/step - loss: 15.5178 - completion_loss: 8.0259 - problems_loss: 7.4919 - val_loss: 14.2933 - val_completion_loss: 8.6149 - val_problems_loss: 5.6784
Epoch 2/1000
3949/3949 [==============================] - 172s 43ms/step - loss: 14.9269 - completion_loss: 7.8407 - problems_loss: 7.0862 - val_loss: 14.1651 - val_completion_loss: 8.5041 - val_problems_loss: 5.6609
Epoch 3/1000
3949/3949 [==============================] - 146s 37ms/step - loss: 14.7649 - completion_loss: 7.8028 - problems_loss: 6.9621 - val_loss: 14.1711 - val_completion_loss: 8.5131 - val_problems_loss: 5.6580
Epoch 4/1000
3949/3949 [==============================] - 140s 35ms/step - loss: 14.6446 - completion_loss: 7.7534 - problems_loss: 6.8912 - val_loss: 14.1486 - val_completion_loss: 8.5072 - val_problems_loss: 5.6414
Epoch 5/1000
3949/3949 [==============================] - 137s 35ms/step - loss: 14.5885 - completion_loss: 7.7343 - problems_loss: 

Epoch 3/1000
3949/3949 [==============================] - 140s 35ms/step - loss: 14.1078 - completion_loss: 7.8375 - problems_loss: 6.2703 - val_loss: 14.0542 - val_completion_loss: 8.2908 - val_problems_loss: 5.7634
Epoch 4/1000
3949/3949 [==============================] - 139s 35ms/step - loss: 14.0116 - completion_loss: 7.8018 - problems_loss: 6.2098 - val_loss: 14.0366 - val_completion_loss: 8.2796 - val_problems_loss: 5.7569
Epoch 5/1000
3949/3949 [==============================] - 139s 35ms/step - loss: 13.9424 - completion_loss: 7.7893 - problems_loss: 6.1531 - val_loss: 14.0278 - val_completion_loss: 8.2841 - val_problems_loss: 5.7437
Epoch 6/1000
3949/3949 [==============================] - 138s 35ms/step - loss: 13.8786 - completion_loss: 7.7699 - problems_loss: 6.1088 - val_loss: 14.0703 - val_completion_loss: 8.3231 - val_problems_loss: 5.7471
Epoch 7/1000
3949/3949 [==============================] - 139s 35ms/step - loss: 13.8579 - completion_loss: 7.7690 - problems_loss: 

Epoch 19/1000
3949/3949 [==============================] - 147s 37ms/step - loss: 14.2593 - completion_loss: 7.6432 - problems_loss: 6.6162 - val_loss: 14.1523 - val_completion_loss: 8.3871 - val_problems_loss: 5.7652
Epoch 1/1000
3949/3949 [==============================] - 155s 38ms/step - loss: 15.4879 - completion_loss: 8.0168 - problems_loss: 7.4710 - val_loss: 14.0805 - val_completion_loss: 8.5369 - val_problems_loss: 5.5436
Epoch 2/1000
3949/3949 [==============================] - 154s 39ms/step - loss: 14.8320 - completion_loss: 7.8449 - problems_loss: 6.9872 - val_loss: 13.9957 - val_completion_loss: 8.4467 - val_problems_loss: 5.5490
Epoch 3/1000
3949/3949 [==============================] - 154s 39ms/step - loss: 14.6824 - completion_loss: 7.7785 - problems_loss: 6.9039 - val_loss: 13.9101 - val_completion_loss: 8.3958 - val_problems_loss: 5.5144
Epoch 4/1000
3949/3949 [==============================] - 152s 39ms/step - loss: 14.5754 - completion_loss: 7.7586 - problems_loss:

In [5]:
# Load the data

exp_norm_map = pd.read_csv('data/experiment_information/exp_norm_map.csv')
results = pd.read_csv('assignments_cross_validation_results.csv')

In [6]:
# Evaluate the results

norm_exp_dict = defaultdict(lambda: 'None')
for n, e in zip(exp_norm_map['normal_id'], exp_norm_map['experiment_id']):
    norm_exp_dict[n] = e
results['experiment_sequence'] = results['target_sequence'].map(norm_exp_dict)

metrics = []
0
# Calculate the metrics for each sequence
for sequence, df in results.groupby('target_sequence'):
    
    completion_target = df['completion_target']
    
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([sequence, 
                    norm_exp_dict[sequence], 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for each fold
for fold, df in results.groupby('fold'):
    
    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([f'fold_{fold}', 
                    'None', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for everything combined
completion_target = results['completion_target']
completion_prediction = results['completion_prediction']
completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_mse = mean_squared_error(completion_target, completion_prediction)

problems_target = results[results['completion_target'] == 1]['problems_target']
problems_prediction = results[results['completion_target'] == 1]['problems_prediction']
problems_mae = mean_absolute_error(problems_target, problems_prediction)
problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_mse = mean_squared_error(problems_target, problems_prediction)

metrics.append(['all_target_data', 
                'None', 
                len(results), 
                completion_auc, 
                completion_acc, 
                completion_r2, 
                completion_ev, 
                completion_mse, 
                problems_mae, 
                problems_mape, 
                problems_r2, 
                problems_ev, 
                problems_mse])

metrics = pd.DataFrame(metrics, 
                       columns=['sequence_id', 
                                'experiment_id', 
                                'sample_size', 
                                'completion_auc', 
                                'completion_acc', 
                                'completion_r2', 
                                'completion_ev', 
                                'completion_mse', 
                                'problems_mae', 
                                'problems_mape', 
                                'problems_r2', 
                                'problems_ev', 
                                'problems_mse'])

metrics.to_csv('assignments_cross_validation_metrics.csv', index=False)

In [7]:
metrics.iloc[-6:]

,sequence_id,experiment_id,sample_size,completion_auc,completion_acc,completion_r2,completion_ev,completion_mse,problems_mae,problems_mape,problems_r2,problems_ev,problems_mse
288,fold_0,None,42114,0.758907,0.772783,0.163067,0.163869,0.156691,1.884888,0.197260,0.039056,0.039107,7.274270
289,fold_1,None,42114,0.765421,0.771430,0.173721,0.175152,0.157298,1.959160,0.215911,0.034879,0.034880,9.209634
290,fold_2,None,42114,0.762735,0.771786,0.171499,0.171524,0.157133,1.983855,0.217319,0.033061,0.033104,9.904209
291,fold_3,None,42113,0.758950,0.764728,0.163658,0.163693,0.160641,1.934242,0.220038,0.037408,0.037735,7.590704
292,fold_4,None,42113,0.759291,0.768931,0.163697,0.164275,0.159083,1.937834,0.204980,0.041874,0.041914,7.797101
293,all_target_data,None,210568,0.760655,0.769932,0.167178,0.167585,0.158169,1.939924,0.211071,0.037096,0.037096,8.354623
